# Example of confidence interval computation

In [ ]:
from __future__ import annotations

import matplotlib.pyplot as plt
import numpy as np
import zfit
from utils import one_minus_cl_plot, plotfitresult, pltdist
from zfit.loss import ExtendedUnbinnedNLL
from zfit.minimize import Minuit

from hepstats.hypotests import ConfidenceInterval
from hepstats.hypotests.calculators import AsymptoticCalculator
from hepstats.hypotests.parameters import POIarray

In [ ]:
plt.rcParams["figure.figsize"] = (9, 8)
plt.rcParams["font.size"] = 16

### Fit of a Gaussian signal over an exponential background:

In [ ]:
bounds = (0.1, 3.0)

# Data and signal

np.random.seed(0)
tau = -2.0
beta = -1 / tau
data = np.random.exponential(beta, 300)
peak = np.random.normal(1.2, 0.1, 80)
data = np.concatenate((data, peak))
data = data[(data > bounds[0]) & (data < bounds[1])]

pltdist(data, bins=80, bounds=bounds)

In [ ]:
obs = zfit.Space("x", limits=bounds)

In [ ]:
mean = zfit.Parameter("mean", 1.2, 0.5, 2.0)
sigma = zfit.Parameter("sigma", 0.1, 0.02, 0.2)
lambda_ = zfit.Parameter("lambda", -2.0, -4.0, -1.0)
Nsig = zfit.Parameter("Nsig", 20.0, -20.0, len(data))
Nbkg = zfit.Parameter("Nbkg", len(data), 0.0, len(data) * 1.1)

In [ ]:
signal = zfit.pdf.Gauss(obs=obs, mu=mean, sigma=sigma).create_extended(Nsig)
background = zfit.pdf.Exponential(obs=obs, lambda_=lambda_).create_extended(Nbkg)
tot_model = zfit.pdf.SumPDF([signal, background])

In [ ]:
# Create the negative log likelihood
data_ = zfit.data.Data.from_numpy(obs=obs, array=data)
nll = ExtendedUnbinnedNLL(model=tot_model, data=data_)

In [ ]:
# Instantiate a minuit minimizer
minimizer = Minuit()

In [ ]:
# minimisation of the loss function
minimum = minimizer.minimize(loss=nll)
minimum.hesse()

In [ ]:
nbins = 80
pltdist(data, nbins, bounds)
plotfitresult(tot_model, bounds, nbins)
plt.xlabel("m [GeV/c$^2$]")
plt.ylabel("number of events")

### Confidence interval

We want to compute the confidence interval of the mean of the Gaussian at 68% confidence level.

In [ ]:
# instantation of the calculator
calculator = AsymptoticCalculator(nll, minimizer)
calculator.bestfit = minimum  # optionnal

In [ ]:
# parameter of interest of the null hypothesis
poinull = POIarray(mean, np.linspace(1.15, 1.26, 100))

In [ ]:
# instantation of the discovery test
ci = ConfidenceInterval(calculator, poinull)

In [ ]:
ci.interval();

In [ ]:
f = plt.figure(figsize=(9, 8))
one_minus_cl_plot(poinull.values, ci.pvalues())
plt.xlabel("mean")
f.savefig("asy_ci.png")